<a href="https://colab.research.google.com/github/zimzion/EYTIP/blob/master/Functional_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>